In [122]:
import requests
import os
import json

In [123]:
url = "https://annotations.allmaps.org/images/2e1d3f991aad6cb4"

In [124]:
df = requests.get(url).json()

opath = "data/input.json"

os.makedirs(os.path.dirname(opath), exist_ok=True)

with open(opath, "w") as f:
    json.dump(df, f, indent=2)

In [127]:
canvas_id = "http://www.example.org/georeferenced-canvas.json"
label = {
    "ja": [
        "東京帝國大學本部構内及農學部建物鳥瞰圖"
    ]
}



def convert_to_1(df):

    target = df["items"][0]["target"]

    service_id = target["service"][0]["@id"] + "/info.json"

    df_i = requests.get(service_id).json()

    opath_i = "data/input_info.json"

    os.makedirs(os.path.dirname(opath_i), exist_ok=True)

    with open(opath_i, "w") as f:
        json.dump(df_i, f, indent=2)

    width = df_i["width"]
    height = df_i["height"]

    features = []

    

    for feature in df["items"][0]["body"]["features"]:

        coordinates = feature["geometry"]["coordinates"]

        coordinates.reverse()

        pixelCoords = feature["properties"]["pixelCoords"]

        line = 50

        xywh = f"{int(pixelCoords[0] - line / 2)},{int(pixelCoords[1] - line / 2)},{line},{line}"

        feature_new = {
            # "id": f"{len(features) + 1}",
            "type": "Feature",
            "properties": {
                "resourceCoords": pixelCoords
            },
            "geometry": {
                "type": "Point",
                "coordinates": coordinates
            },
            # "xywh": xywh
        }

        features.append(feature_new)

    itemAnnotationPageId = "http://www.example.org/contentPage.json"

    itemAnnotationAnnotationId = "http://www.example.org/content.json"

    itemAnnotationAnnotation = {
        "id": itemAnnotationAnnotationId,
        "type": "Annotation",
        "motivation": "painting",
        "body": {
            "id": f"{df_i['@id']}/full/full/0/default.jpg",
            "type": "Image",
            "format": "image/jpeg",
            "height": height,
            "width": width,
            "service": [
                {
                    "id": df_i["@id"],
                    "type": "ImageService2"
                }
            ]
        },
        "target": canvas_id
    }

    itemAnnotationPage = {
        "id": itemAnnotationPageId,
        "type": "AnnotationPage",
        "items": [
            itemAnnotationAnnotation
        ]
    }

    annotationAnnotationId = "http://www.example.org/canvas-annotation.json"

    featureCollectionId = "http://www.example.org/feature-collection.json"

    annotationAnnotation = {
        "id": annotationAnnotationId,
        "type": "Annotation",
        "motivation": "georeferencing",
        "target": canvas_id,
        "body": {
            "id": featureCollectionId,
            "type": "FeatureCollection",
            "transformation": {
                "type": "polynomial",
                "options": {
                    "order": 1
                }
            },
            "features": features
        }
    }

    annotationAnnotationPageId = "http://www.example.org/annotationPage.json"

    annotationAnnotationPage = {
                "id": annotationAnnotationPageId,
                "type": "AnnotationPage",
                "items": [
                    annotationAnnotation
                ]
            }

    res = {
        "@context": [
            "http://iiif.io/api/extension/georef/1/context.json",
            "http://iiif.io/api/presentation/3/context.json"
        ],
        "id": canvas_id,

        "type": "Canvas",
        "label": label,
        "height": height,
        "width": width,

        "items": [
            itemAnnotationPage
        ],

        "annotations": [
            annotationAnnotationPage
        ]
    }

    opath_r = "data/output2.json"

    os.makedirs(os.path.dirname(opath_r), exist_ok=True)

    with open(opath_r, "w") as f:
        json.dump(res, f, indent=2, ensure_ascii=False)


In [128]:
convert_to_1(df)